In [3]:
# import TensorFlow as TF
import pandas as pd
import math
import matplotlib.pylab as plt
import numpy as np

from sklearn import datasets
from sklearn.impute import SimpleImputer # used for handling missing data


In [4]:

df = pd.read_csv('data.csv')
data_top = df.head()
copy = df.copy() 

# 1 - Remove rows where the target column (Scope1) is NaN.

copy = copy[copy['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'].notna()] # drop all rows with NaN in target column

copy = copy.reset_index() 



# 2 - Replace all NaN from the attribute columns and replace for mean of that column

#copy = copy.fillna(copy.mean())
#col_names = copy.columns
#sub2[col_names].fillna((sub2[col_names].mean()), inplace=True)

# 3 - Group rows by organization ID

# org_ids = list(copy['GTAP country'])
# list_set = set(org_ids)
# unique = list(list_set)

# copy = copy.groupby(['GTAP country'])

# dictionary = {} 

# for item in unique:
#     df_id = copy.get_group(item)
#     dictionary[item] = df_id


def create_dict(target_var, df): 
    org_ids = list(df[target_var]) 
    list_set = set(org_ids) # get unique elements
    unique = list(list_set) # place them in a list 

    df = df.groupby([target_var]) 

    dictionary = {} 

    for item in unique:
        df_id = df.get_group(item)
        dictionary[item] = df_id
    
    return dictionary
    

# dictionaries per target variable:

GTAP_sector_dict = create_dict('GTAP sector', copy)

GTAP_country_dict = create_dict('GTAP country', copy)



key_list = list(GTAP_sector_dict.keys())

print(key_list)

#print(GTAP_sector_dict)

# in the dictionary, we want to replace all the NaN of the values for the means of those columns.


# # check missing values >> columns with lots of missing values are not useful. 
# missing_values_df = copy.isnull().sum().sort_values(ascending=False).head()
# print(missing_values_df)

# # replace NaN in column for mean of column with Imputer 
# imp = SimpleImputer(missing_values='NaN', strategy='mean')
# imp = imp.fit(copy)



#print(replace_NaN(GTAP_sector_dict))    



['Motor vehicles and parts', 'Ferrous metals', 'Recreational and other services', 'Human health and social work activities', 'Null', 'Electrical equipment', 'Electricity', 'Basic pharmaceutical products', 'Bovine cattle, sheep and goats, horses', 'Machinery and equipment nec', 'Beverages and tobacco products', 'Mineral products nec', 'Coal', 'Construction', 'Paper products, publishing', 'Wheat', 'Air transport', 'Business services nec', 'Animal products nec', 'Real estate activities', 'Chemical products', 'Insurance', 'Crops nec', 'Wearing apparel', 'Oil', 'Transport equipment nec', 'Warehousing and support activities', 'Metal products', 'Gas', 'None', 'Water transport', 'Metals nec', 'Rubber and plastic products', 'Petroleum, coal products', 'Communication', 'Transport nec', 'Wood products', 'Processed rice', 'Forestry', 'Sugar cane, sugar beet', 'Manufactures nec', 'Trade', 'Fishing', 'Vegetables, fruit, nuts', 'Textiles', 'Financial services nec', 'Computer, electronic and optical p

In [6]:
# function for filling NaN's
def fill_nan(df, column_name):
    mean_value=df[column_name].mean()
    df[column_name].fillna(value=mean_value, inplace=True)
    return df


0        7.687916
1        7.687916
2        7.687916
3        0.793000
4        7.687916
           ...   
25081    7.687916
25082    7.687916
25083    7.687916
25084    7.687916
25085    7.687916
Name: EmissionDataPoints.AccidentalSpills, Length: 25086, dtype: float64
0        7.687916
1        7.687916
2        7.687916
3        0.793000
4        7.687916
           ...   
25081    7.687916
25082    7.687916
25083    7.687916
25084    7.687916
25085    7.687916
Name: EmissionDataPoints.AccidentalSpills, Length: 25086, dtype: float64
